# NYC 311 Service Requests Data Pipeline

*Library + jobs to fetch, cache, and analyze NYC 311 service request data.*

# As Jobs

## LTP Pipeline

In [ ]:
from pathlib import Path

# Per-user, cross-platform base under the home dir
# retrieve for one day only:
dir_base = Path.home() / "TASK_HBC_TSY"

%cd -q ..
!python -m hbc.jobs.dispatch \
  --job-name=job_fetch_nyc_open_data_311_service_requests \
  --as-of=20091231 \
  --dir-base={dir_base} \
  --incremental=True \
  --log-level=INFO
%cd -q notebooks/

In [ ]:
# restore cache integrity for the last missing dates
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_fetch_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --incremental=false \
      --log-level=INFO \
      --last-missing-dates=5
%cd -q notebooks/

## Analytics

In [ ]:
%cd -q ..
!python -m hbc.jobs.dispatch  \
      --job-name=job_analyse_nyc_open_data_311_service_requests \
      --as-of=20091231 \
      --log-level=INFO \
      --n-worst=10 \
      --n-best=10 \
      --n-days=10
%cd -q notebooks/

***

# As Library 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Imports

In [ ]:
import sys
from pathlib import Path

p = str(Path.cwd().parent) # one dir up
if p not in sys.path:
    sys.path.insert(0, p)

import pandas as pd
import os
import numpy as np
import logging
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## Api

In [1]:
from hbc import app_context, DataContainer, utils as ul

from hbc.quant.analysis import AnalyticalEngine
from hbc.quant.plots import PlotEngine

ModuleNotFoundError: No module named 'hbc'

In [ ]:
app_context

## Logging

In [6]:
import logging
logger = logging.getLogger()

In [7]:
# Console-only (no file writes):
ul.conf_log(level=logging.DEBUG, console=True, file=False, reset_handlers=True)

# File-only (no console output at all):
ul.conf_log(level=logging.INFO, console=False, file=True, reset_handlers=True)

# Both:
ul.conf_log(level=logging.DEBUG, console=True, file=True, reset_handlers=True)

Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/hbc_job_generic.txt
Log file: /var/folders/jj/dn25brln45j26cvj4y_lgbzr0000gn/T/hbc_nyc_dp/LOGS/hbc_job_generic.txt


<RootLogger root (DEBUG)>

# Cache To SQLLite DataBase Via Asp.Net Rest Api with EF Core

In [13]:
import hbc
from hbc import DataContainer
from hbc.ltp.persistence.db import SqlLiteDataBase

In [14]:
logger.setLevel(logging.INFO)

In [15]:
import os
from pathlib import Path

dc = DataContainer("nyc_open_data_311_customer_satisfaction_survey")
dc.get(limit=100)

2025-12-25 09:01:16 fetch_nycopen.py       71 INFO  root    : Fetched 100 rows
2025-12-25 09:01:16 base.py                52 INFO  root    : using validator: ValidatorGeneric
2025-12-25 09:01:16 base.py                53 INFO  root    : cleaning...
2025-12-25 09:01:16 base.py                56 INFO  root    : normalizing...
2025-12-25 09:01:16 base.py                59 INFO  root    : validating...
2025-12-25 09:01:16 base.py                62 INFO  root    : finalizing...


TypeError: DataContainer._add_hbc_unique_key() takes 1 positional argument but 2 were given

In [11]:
dc.to_cache()
dc.from_cache(query=f'select * from {dc.moniker}').shape

2025-12-25 08:59:22 rest.py                80 INFO  root    : Posting batch 1-100/100 to http://localhost:5047/nyc_open_data_311_customer_satisfaction_survey/batch (verify=False)
2025-12-25 08:59:23 rest.py               105 INFO  root    : Synced 100 rows via batch API


(100, 15)

In [12]:
dc.from_cache().head(2)

,id,hbc_unique_key,year,campaign,channel,survey_type,start_time,completion_time,survey_language,overall_satisfaction,wait_time,agent_customer_service,agent_job_knowledge,answer_satisfaction,nps
0,1,0193950b24c0fd7540f81ebd67ef7eec9dd63ca7,2021,Campaign 5,NYC311 Call Center,Customer Satisfaction (CSAT) Survey,2021-12-08 15:09:24,2021-12-08 15:18:59,English (United States)‎,Agree,Agree,Agree,Agree,Disagree,7
1,2,aa41a7cd3809f37af6dc9100c377e5139e8da7b3,2021,Campaign 5,NYC311 Call Center,Customer Satisfaction (CSAT) Survey,2021-12-13 09:17:09,2021-12-13 09:19:31,English (United States)‎,Agree,Agree,Agree,Neutral,Neutral,7


In [27]:
db = SqlLiteDataBase()

In [20]:
db.execute(f'delete from {dc.moniker}')

400

In [22]:
db.all_dbs

['main:/Users/alexandershubert/git/hbc_tsy_enhanced/hbc_db/hbc.db']

In [23]:
db.all_tables

['main:__EFMigrationsHistory',
 'main:nyc_open_data_311_customer_satisfaction_survey',
 'main:sqlite_sequence']

****

## DataContainer

In [ ]:
dc = DataContainer('nyc_open_data_311_service_requests')

In [ ]:
# we retrieve 100 rows
dc.get()

In [ ]:
dc.get(where=f"created_date = '{ul.date_as_iso_format(ul.str_as_date('20091231'))}' ")

In [ ]:
dc.to_cache()

In [ ]:
dc.from_cache().head()

In [ ]:
', '.join(dc.all_cached_dates)

## Analytics

In [ ]:
df = dc.df

In [ ]:
cols = ul.cols_as_named_tuple(df)

In [ ]:
df = df[~df[cols.DROP_FLAG]]

In [ ]:
df["hbc_days_to_close"] = (
    pd.to_datetime(df[cols.closed_date])
    - pd.to_datetime(df[cols.created_date])
).dt.days.astype("Int64")
cols = ul.cols_as_named_tuple(df)

In [ ]:
m = df[cols.hbc_days_to_close] == 0
df_closed_not_same_day = df[~m]

In [ ]:
path = ul.path_to_str(
                ul.mk_dir(ul.get_dir_analytics() / "plots")
                / "closed_requests_by_location.html"
            )
_ = PlotEngine.plot_geo_map(
            df=df_closed_not_same_day,
            col_latitude=cols.latitude,
            col_longitude=cols.longitude,
            aggregation="count",
            round_precision=3,
            cluster=True,
            start_zoom=11,
            tiles="CartoDB positron",
            savepath= path
        )
print(path)

In [ ]:
# by agency
res = AnalyticalEngine.descriptive_stats(
    n_best=10,
    n_worst=10,
    df=df_closed_not_same_day,
    col_metric=cols.hbc_days_to_close,
    group=[
        cols.agency,
        cols.agency_name,
    ],
)


In [ ]:
res.keys()

In [ ]:
res['worst']

### new dataset: nyc_open_data_311_call_center_inquiry:

In [ ]:
dc = DataContainer('nyc_open_data_311_call_center_inquiry')

In [ ]:
dc.get()

In [ ]:
dc.get(where=f"agency='NYPD'", limit=250)

In [ ]:
dc.df.shape

In [ ]:
dc.df.head()

In [ ]:
dc.df.head()

In [ ]:
dc.get(where=f"date = '{ul.date_as_iso_format(ul.str_as_date('2010-01-03'))}'")

In [ ]:
dc.df.head()

### new_dataset: nyc_open_data_311_customer_satisfaction_survey

In [ ]:
dc = DataContainer('nyc_open_data_311_customer_satisfaction_survey')

In [ ]:
dc.get()

In [ ]:
dc.get(where=f"answer_satisfaction='Neutral'")

In [ ]:
dc.df.head()